#  ПРОЕКТ "Рынок заведений общественного питания Москвы"

**Описание проекта:**

Инвесторы из фонда «Shut Up and Take My Money» решили попробовать себя в новой области и открыть заведение общественного питания в Москве. Заказчики ещё не знают, что это будет за место: кафе, ресторан, пиццерия, паб или бар, — и какими будут расположение, меню и цены.

Для начала они просят вас — аналитика — подготовить исследование рынка Москвы, найти интересные особенности и презентовать полученные результаты, которые в будущем помогут в выборе подходящего инвесторам места. 

Нам доступен датасет с заведениями общественного питания Москвы, составленный на основе данных сервисов Яндекс Карты и Яндекс Бизнес на лето 2022 года. Информация, размещённая в сервисе Яндекс Бизнес, могла быть добавлена пользователями или найдена в общедоступных источниках. Она носит исключительно справочный характер.

**Описание данных:**

- name — название заведения;
- address — адрес заведения;
- category — категория заведения, например «кафе», «пиццерия» или «кофейня»;
- hours — информация о днях и часах работы;
- lat — широта географической точки, в которой находится заведение;
- lng — долгота географической точки, в которой находится заведение;
- rating — рейтинг заведения по оценкам пользователей в Яндекс Картах (высшая оценка — 5.0);
- price — категория цен в заведении, например «средние», «ниже среднего», «выше среднего» и так далее;
- avg_bill — строка, которая хранит среднюю стоимость заказа в виде диапазона, например:

«Средний счёт: 1000–1500 ₽»;

«Цена чашки капучино: 130–220 ₽»;

«Цена бокала пива: 400–600 ₽».
и так далее;

- middle_avg_bill — число с оценкой среднего чека, которое указано только для значений из столбца avg_bill, начинающихся с подстроки «Средний счёт»:

 Если в строке указан ценовой диапазон из двух значений, в столбец войдёт медиана этих двух значений.
 
 Если в строке указано одно число — цена без диапазона, то в столбец войдёт это число.
 
 Если значения нет или оно не начинается с подстроки «Средний счёт», то в столбец ничего не войдёт.
- middle_coffee_cup — число с оценкой одной чашки капучино, которое указано только для значений из столбца avg_bill, начинающихся с подстроки «Цена одной чашки капучино»:

Если в строке указан ценовой диапазон из двух значений, в столбец войдёт медиана этих двух значений.
Если в строке указано одно число — цена без диапазона, то в столбец войдёт это число.
Если значения нет или оно не начинается с подстроки «Цена одной чашки капучино», то в столбец ничего не войдёт.
- chain — число, выраженное 0 или 1, которое показывает, является ли заведение сетевым (для маленьких сетей могут встречаться ошибки):

0 — заведение не является сетевым,
1 — заведение является сетевым;
- district — административный район, в котором находится заведение, например Центральный административный округ;
- seats — количество посадочных мест.

**План исследования:**

- 1.Обзор данных;
- 2.Предобработка данных;
- 3.Анализ данных;
- 4.Детализация исследования: открытие кофейни;
- 5.Презентация результатов.

**Шаг 1. Загрузим данные и изучим общую информацию**


In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt, timedelta
from matplotlib import pyplot as plt
from plotly import graph_objects as go
import plotly.express as px
import seaborn as sns
from plotly import graph_objects as go
import json
from folium import Map, Choropleth, Marker
from folium.plugins import MarkerCluster
import folium
from folium import plugins
from folium.plugins.heat_map import HeatMap

In [2]:
places = pd.read_csv('/datasets/moscow_places.csv')
places.head()

,name,category,address,district,hours,lat,lng,rating,price,avg_bill,middle_avg_bill,middle_coffee_cup,chain,seats
0,WoWфли,кафе,"Москва, улица Дыбенко, 7/1",Северный административный округ,"ежедневно, 10:00–22:00",55.878494,37.478860,5.0,NaN,NaN,NaN,NaN,0,NaN
1,Четыре комнаты,ресторан,"Москва, улица Дыбенко, 36, корп. 1",Северный административный округ,"ежедневно, 10:00–22:00",55.875801,37.484479,4.5,выше среднего,Средний счёт:1500–1600 ₽,1550.0,NaN,0,4.0
2,Хазри,кафе,"Москва, Клязьминская улица, 15",Северный административный округ,"пн-чт 11:00–02:00; пт,сб 11:00–05:00; вс 11:00...",55.889146,37.525901,4.6,средние,Средний счёт:от 1000 ₽,1000.0,NaN,0,45.0
3,Dormouse Coffee Shop,кофейня,"Москва, улица Маршала Федоренко, 12",Северный административный округ,"ежедневно, 09:00–22:00",55.881608,37.488860,5.0,NaN,Цена чашки капучино:155–185 ₽,NaN,170.0,0,NaN
4,Иль Марко,пиццерия,"Москва, Правобережная улица, 1Б",Северный административный округ,"ежедневно, 10:00–22:00",55.881166,37.449357,5.0,средние,Средний счёт:400–600 ₽,500.0,NaN,1,148.0


In [3]:
places.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8406 entries, 0 to 8405
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   name               8406 non-null   object 
 1   category           8406 non-null   object 
 2   address            8406 non-null   object 
 3   district           8406 non-null   object 
 4   hours              7870 non-null   object 
 5   lat                8406 non-null   float64
 6   lng                8406 non-null   float64
 7   rating             8406 non-null   float64
 8   price              3315 non-null   object 
 9   avg_bill           3816 non-null   object 
 10  middle_avg_bill    3149 non-null   float64
 11  middle_coffee_cup  535 non-null    float64
 12  chain              8406 non-null   int64  
 13  seats              4795 non-null   float64
dtypes: float64(6), int64(1), object(7)
memory usage: 919.5+ KB


In [4]:
places['name'].nunique()

5614

В датасете о заведениях общественного питания Москвы предствлено 8406 строк. Всего 5614 названий заведений, остальные либо дубликаты либо заведения со идентичным названием.
Тип данных соответсвует действительности. Имеются пропуски в столбцах hours, price, avg_bill, middle_avg_bill, middle_coffee_cup,seats. Требуется обработка данных.


**Шаг 2. Предобработка данных**

In [5]:
places.duplicated().sum()

0

In [6]:
#проверим на неявные дубликаты, приведем названия все названия заведений к нижнему регистру
places['name'] = places['name'].str.upper()
places['name'].nunique()

5512

In [7]:
places['category'].unique()

array(['кафе', 'ресторан', 'кофейня', 'пиццерия', 'бар,паб',
       'быстрое питание', 'булочная', 'столовая'], dtype=object)

In [8]:
#проверим дубликаты по паре "название заведения - адрес"
places.duplicated(subset=['name', 'address']).sum()

3

In [9]:
#удалим дубликаты
places = places.drop_duplicates(subset=['name', 'address'])

In [10]:
places.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8403 entries, 0 to 8405
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   name               8403 non-null   object 
 1   category           8403 non-null   object 
 2   address            8403 non-null   object 
 3   district           8403 non-null   object 
 4   hours              7868 non-null   object 
 5   lat                8403 non-null   float64
 6   lng                8403 non-null   float64
 7   rating             8403 non-null   float64
 8   price              3315 non-null   object 
 9   avg_bill           3816 non-null   object 
 10  middle_avg_bill    3149 non-null   float64
 11  middle_coffee_cup  535 non-null    float64
 12  chain              8403 non-null   int64  
 13  seats              4792 non-null   float64
dtypes: float64(6), int64(1), object(7)
memory usage: 984.7+ KB


In [11]:
# количество пропущенных значений
places.isna().sum()

name                    0
category                0
address                 0
district                0
hours                 535
lat                     0
lng                     0
rating                  0
price                5088
avg_bill             4587
middle_avg_bill      5254
middle_coffee_cup    7868
chain                   0
seats                3611
dtype: int64

In [ ]:
pd.DataFrame(round(places.isna().mean()*100,1)).style.background_gradient('coolwarm')

,0
name,0.000000
category,0.000000
address,0.000000
district,0.000000
hours,6.400000
lat,0.000000
lng,0.000000
rating,0.000000
price,60.500000
avg_bill,54.600000


In [ ]:
#создадим колонку с названием улицы из столбца с адресом
def street_name(row):
    address = row['address'].split(', ')
    street_name = address[1]
    return street_name
places['street'] = places.apply(street_name, axis=1)

In [ ]:
# заменим пропуски в колонке hours на unknown
places['hours'] = places['hours'].fillna('unknown')

In [ ]:
#создадим столбец обозначающий, что заведение работает круглосуточно и ежедневно is_24/7 
def hours_24(row):
    hours = row['hours'].split(', ')
    if hours[0] == 'ежедневно' and hours[1] == 'круглосуточно':
        return True
    else:
        return False
places['is_24/7'] = places.apply(hours_24, axis=1)

In [ ]:
places.sample()

**Вывод по шагу 2:**
- **Пропуски:**
- - Были найдены пропуски в столбцах: - hours (6.4%), seats (43%), price(60,5%), avg__bill (54,6%), middle_avg_bill (62,5%) самое большое количество пропусков в middle_coffee_cup (93.6%). Заполнять пропуски в данных со существеным количеством пропусков средними или медианными значениями будет некорректно и сильно исказит результаты исследования, поэтому оставим все, как есть.
- - в колонке hours пропуски заменили на 'unknown'.
- **Дубликаты:**
- - Явных дубликатов найдено не было.
- - Были найдены и удалены 3 дубликата по паре значений 'название заведения - адрес'
- - Количество уникальных значений в столбце name было сокращено до 5512
- **Новые столбцы:**
- - Были созданы столбцы 'street' с названием улицы и 'is_24/7' с обозначением круглосуточной и ежедневной работы.

**Шаг 3. Анализ данных**

**3.1 Какие категории заведений представлены в данных? Исследуем количество объектов общественного питания по категориям: рестораны, кофейни, пиццерии, бары и так далее. Построим визуализации. Ответим на вопрос о распределении заведений по категориям.**

In [ ]:
category_count = places['category'].value_counts().reset_index()
category_count.columns = ['type', 'total']
category_count

In [ ]:
plt.figure(figsize=(13, 7))
sns.barplot(x='type', y='total', data=category_count)
plt.title('Количество заведений в каждой категории')
plt.xlabel('Количество')
plt.ylabel('Категория заведения')
plt.show()

Мы видим 8 категорий заведений.Больше всего заведений - кафе, меньше всего - булочных.

**3.2 Исследуйте количество посадочных мест в местах по категориям: рестораны, кофейни, пиццерии, бары и так далее. Постройте визуализации. Проанализируйте результаты и сделайте вывод.**

In [ ]:
plt.figure(figsize=(15, 7))
sns.boxplot(x='seats', y='category', data=places) 
plt.xlabel('Количество посадочных мест')
plt.ylabel('Тип заведения')
plt.xlim()
plt.grid()
plt.title('Распределение посадочных мест по типам заведений')
plt.show()

In [ ]:
places['seats'].describe()

In [ ]:
seats_count = places.groupby('category').agg({'seats' : 'median','name' : 'count'}).reset_index()
seats_count.columns = ['category', 'median', 'count']
seats_count = seats_count.sort_values(by='median', ascending=False)
seats_count

In [ ]:
plt.figure(figsize=(10, 7))
sns.barplot(x='category', y='median', data=seats_count)
plt.xlabel('Количество посадочных мест')
plt.ylabel('Категория заведения')
plt.title('Медианное распределение посадочных мест по категориям заведений')
plt.show()

Больше всего посадочных мест в ресторанах - 86(медианное значение). Далее идут пабы и бары - 82 места. У кофеен и столовых медианное количество мест 80 и 75 соотвественно. В фастфудах обычно 65 мест, в кафе и пиццериях 60 и 55 мест. Меньше всего мест в булочных - 50.

**3.3 Рассмотрим и изобразим соотношение сетевых и несетевых заведений в датасете. Каких заведений больше?**

In [ ]:
#посчитаем количество 
is_chain = places['chain'].value_counts().reset_index()
is_chain .columns = ['chain', 'count']
is_chain 

In [ ]:
fig = go.Figure(data=[go.Pie(labels=is_chain['chain'].map({1: 'Доля сетевых заведений', 0: 'Доля несетевых заведений'}), 
                             values=is_chain['count'],                              
                             title='Распределение сетевых и несетевых заведений')])
fig.update_traces(marker=dict(colors=['#612147','#FFA900']))
fig.show()

Несетевых заведений больше - 5200 (61.9%), сетевых заведений - 3203(38.1%).

**3.4 Какие категории заведений чаще являются сетевыми? Исследуем данные и ответим на вопрос графиком.**

In [ ]:
# категории сетевых заведений
chain = (places
         .pivot_table(index='category', values='chain', aggfunc=('count', 'sum'))
         .sort_values(by='count', ascending=False)
         .reset_index()
        )
chain.columns = ['category', 'total_count', 'chain_count']
#добавим столбец с количеством несетевых заведений
chain['unchain_count'] = chain['total_count'] - chain['chain_count']
#посчитаем доли сетевых и несетевых заведений в %
chain['chain_%'] = round(chain['chain_count']/chain['total_count'] * 100,1)
chain['unchain_%'] = round(chain['unchain_count']/chain['total_count'] * 100,1)
chain

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x = chain['category'], y = chain['unchain_count'], marker_color='#612147', name='Несетевые заведения', 
                     text=chain['unchain_%'].apply(lambda x: '{0:1.2f}%'.format(x))))
fig.add_trace(go.Bar(x = chain['category'], y = chain['chain_count'], marker_color='#FFA900', name='Сетевые заведения', 
                     text=chain['chain_%'].apply(lambda x: '{0:1.2f}%'.format(x))))
fig.update_layout(
    title='Распределение сетевых и несетевых заведений по категориям',
    xaxis_title='Категория заведения',
    yaxis_title='Количество заведений'
)

fig.show()

Больше всех сетями являются булочные заведения - 61.3%, меньше всех пабы и бары - 22%. Кафе и рестораны чаще являются несетевым заведением - 67,2% и 64,3%. Кофеен почти половина сетей. 78% пабов и баров являюся нестевыми. Заведения категории 'фастфуд' чаще несетевое также как и столовые.

**3.5 Сгруппируем данные по названиям заведений и найдем топ-15 популярных сетей в Москве. Под популярностью понимается количество заведений этой сети в регионе. Постройте подходящую для такой информации визуализацию. Знакомы ли вам эти сети? Есть ли какой-то признак, который их объединяет? К какой категории заведений они относятся?**

In [ ]:
places['name'].value_counts().head(20)

In [ ]:
#уберем названия заведений, которые объединили с в название категории. Яндекс Лавку также исключим из списка, так как это онлайн-сервис
places_chain = places.query('name not in ["КАФЕ", "ХИНКАЛЬНАЯ", "ШАУРМА", "ЧАЙХАНА", "РЕСТОРАН", "СТОЛОВАЯ", "ЯНДЕКС ЛАВКА"]')
top_15_chain = places_chain[['name', 'category']].value_counts().head(15).reset_index()
top_15_chain.columns = ['name', 'category', 'count']
top_15_chain

In [ ]:
fig = px.bar(top_15_chain, x='count', y='name', color='category')
fig.update_layout(title='Топ-15 сетевых заведений',
                   xaxis_title='Количество заведений сети',
                   yaxis_title='Название сети',
                   yaxis={'categoryorder':'total ascending'})
fig.show()

Из графика видно, что самым крупным сетевым заведением является Шоколадница - 119 кафе. Пиццерии Доминос и Додо занимают 2 и 3 места, 77 и 74 заведения соответсвенно. Самыми скромными сетями в нашем рейтинге будут Крошка Картошка и Cinnabon - по 20 точек в каждом. В общем самые распространенные сетевые категории - это кафе, кофейни, булочные, пиццерии.

**3.6 Какие административные районы Москвы присутствуют в датасете? Отобразим общее количество заведений и количество заведений каждой категории по районам. Попробуем проиллюстрировать эту информацию одним графиком.**

In [ ]:
places['district'].value_counts()

In [ ]:
district = (places
                  .pivot_table(index='district', values='name', columns='category', aggfunc=('count'))
                  .reset_index()
                  )
district['total'] = district.loc[:, 'бар,паб':'столовая'].sum(axis=1)
district = district.sort_values(by='total', ascending=False)
district

In [ ]:
fig = px.bar(district, x=['бар,паб', 'булочная', 'быстрое питание', 'кафе', 'кофейня', 'пиццерия', 'ресторан', 'столовая'], 
             y='district'
             )
fig.update_layout(title='Распределение категорий заведений Москвы по кругам',
                   xaxis_title='Округ',
                   yaxis_title='Количество заведений')
fig.show()

По графику можно увидеть, что в Центральном округе больше всего заведений - 2242, среди которых подавляющее число это рестораны. На Северо-Западе Москвы оказалось меньше всего заведений - 409. В остальных округах число заведений примерно одинаковое от 709 до 899.

**3.7 Визуализируем распределение средних рейтингов по категориям заведений. Сильно ли различаются усреднённые рейтинги в разных типах общепита?**

In [ ]:
rating = (places
         .pivot_table(index='category', values='rating', aggfunc='mean')
         .sort_values(by='rating', ascending=False)
         .reset_index())
rating['rating'] = round(rating['rating'],2)
rating

In [ ]:
  
plt.figure(figsize=(10, 7))
sns.barplot(x='category', y='rating', data=rating)
plt.ylim(4, 4.5)
plt.xlabel('Категория заведения')
plt.ylabel('Рейтинг')
plt.title('Распределение средних рейтингов по категориям')
plt.show()

In [ ]:
plt.figure(figsize=(12, 7))

sort_median = (places.loc[:,['rating', 'category']]
    .groupby(['category'])
    .median() 
    .sort_values(by='rating')
          )

sns.boxplot(x='rating', y='category', data=places, order=sort_median.index)
plt.xlabel('Рейтинг')
plt.ylabel('Категория заведения')
plt.xlim()
plt.grid()
plt.title('Распределение рейтингов по категориям заведений')
plt.show()

Наивысший рейтинг у пабов и баров (4.39), следом идут пиццерии (4,30), а за ними рестораны (4,29). У кофеен и булочных примерно поровну 4.28 и 4.27 соответственно. У столовых 4.21, у кафе 4.12. Фастфуды же обладают самым низким рейтингом - 4.05.

**3.8 Построим фоновую картограмму (хороплет) со средним рейтингом заведений каждого района.**

In [ ]:
places_rating = (places
             .groupby('district')['rating'].mean()
             .round(2)
             .sort_values(ascending=False)
             .reset_index()
            )
places_rating

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x = places_rating['district'], y = places_rating['rating']))
fig.update_layout(
    title='Распределение средних рейтингов по категориям',
    xaxis_title='Категория заведения',
    yaxis_title='Средний рейтинг')
fig.show()

In [ ]:
with open('/datasets/admin_level_geomap.geojson', 'r') as f:
    geo_json = json.load(f)

In [ ]:
geomap = '/datasets/admin_level_geomap.geojson'

In [ ]:
moscow_lat, moscow_lng = 55.751244, 37.618423

In [ ]:
map_moscow = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

In [ ]:
Choropleth(
    geo_data=geomap,
    data=places_rating,
    columns=['district', 'rating'],
    key_on='feature.name',
    fill_color='YlOrBr',
    fill_opacity=0.8,
    legend_name='Средний рейтинг заведений по районам',
).add_to(map_moscow)


In [ ]:
map_moscow

Самый высокий средний рейтинг у заведений из Центрального округа - 4.38, самый низкий из Юго-Восточного округа - 4.10. В Северном округе средняя оценка заведений - 4.24. В остальных округах рейтинг в интервале от 4.15 до 4.21.

**3.9 Отобразим все заведения датасета на карте с помощью кластеров.**

In [ ]:
moscow_lat, moscow_lng = 55.751244, 37.618423

msc_map = Map(location=[moscow_lat, moscow_lng], zoom_start=10)
marker_cluster = MarkerCluster().add_to(msc_map)

def create_clusters(row):
    Marker(
        [row['lat'], row['lng']],
        popup=f"{row['name']} {row['rating']}",
    ).add_to(marker_cluster)

places.apply(create_clusters, axis=1)

msc_map

In [ ]:
#тепловая карта заведений

msc_heatmap = folium.Map([55.751244, 37.618423], zoom_start=10)
heatmap = places[['lat','lng']]
msc_heatmap.add_child(plugins.HeatMap(heatmap, radius=14))
msc_heatmap   

Как уже раннее было сказано, в центре больше всего заведений, в остальных округах их количество существенно меньше.

**3.10 Найдем топ-15 улиц по количеству заведений. Построим график распределения количества заведений и их категорий по этим улицам.**

In [ ]:
top_15_street = places['street'].value_counts().head(15).reset_index()
top_15_street.columns = ['street', 'total']
top_15_street

In [ ]:
street_count = places.pivot_table(index='street', values='name', columns='category', aggfunc='count')
street_count

In [ ]:
top_15_street_count = top_15_street.merge(street_count, how='inner', on='street')
top_15_street_count

In [ ]:
fig = px.bar(top_15_street_count, x='street', 
             y=['бар,паб', 'булочная', 'быстрое питание', 'кафе', 'кофейня', 'пиццерия', 'ресторан', 'столовая']
             )
fig.update_layout(title='Топ-15 улиц по количеству заведений',
                   xaxis_title='Название улицы',
                   yaxis_title='Количество заведений')
fig.show()

Из исследования видно, что больше всего заведений находятся на больших улицах: шоссе, проспектах... Самая "густонаселенная" среди них это проспект Мира - 183 заведения. Пятницкая улица, хоть и небольшая по сравнению с шоссе, тем не менее вошла в наш рейтинг, там 48 заведений - наименьший показатель.

**3.11 Найдем улицы, на которых находится только один объект общепита.**

In [ ]:
one_street = places.groupby('street')['name'].count().reset_index().query('name == 1')
one_street.rename(columns={'name':'count'}, inplace=True)
one_street

In [ ]:
df_category = places[['category', 'street']].reset_index()
df_category

In [ ]:
#объединим две таблицы и посчитаем, сколько улиц с одним объектом в каждом категории
category_street = one_street.merge(df_category, how='inner', on='street')
category_street = (category_street
                   .groupby('category')['street'].count()
                   .sort_values(ascending=False)
                   .reset_index()
                  )
category_street

In [ ]:
plt.figure(figsize=(10, 7))
sns.barplot(x='category', y='street', data=category_street)
plt.xlabel('категория заведения')
plt.ylabel('Количество улиц')
plt.title('Количество улиц с одним заведением общепита по категориям ')
plt.show()

Видно, что одиночных кафе больше всех с большим отрывом на втором месте рестораны а за ними кофейни, количество заведений в остальных категорях сильно меньше начиная с баров и пабов.


In [ ]:
df_district = places[['district', 'street']].reset_index()
df_district

In [ ]:
district_street = one_street.merge(df_district, how='inner', on='street')
district_street = (district_street
                   .groupby('district')['street'].count()
                   .sort_values(ascending=False)
                   .reset_index()
                  )
district_street

In [ ]:
plt.figure(figsize=(15, 7))
sns.barplot(x='street', y='district', data=district_street) 
plt.xlabel('Количество улиц')
plt.ylabel('Округ')
plt.grid()
plt.title('Количество улиц с одним заведением общепита по округам')
plt.show()

Всего на 458 улицах Москвы располагается по одному заведению общепита, кафе среди них встречаются чаще всего - 160 точек. Большинство улиц находятся в центре. Меньше всего улиц с одной точкой общественного питания в Северо-Западном и Юго-Западном округах. В остальных округах города распределение улиц с одним заведением общепита равномерное.

**3.12 Рассчитаем средний чек для каждого округа Москвы. Построим фоновую картограмму (хороплет) с полученными значениями для каждого района. Проанализируем цены в центральном административном округе и других. Как удалённость от центра влияет на цены в заведениях?**

In [ ]:
avg_bill = (places
            .groupby('district')['middle_avg_bill'].median()
            .reset_index()
            .sort_values(by = 'middle_avg_bill', ascending=False)
           )
avg_bill

In [ ]:
plt.figure(figsize=(15, 7))
sns.barplot(x='middle_avg_bill', y='district', data=avg_bill) 
plt.xlabel('Средний чек')
plt.ylabel('Округ')
plt.grid()
plt.title('Средний чек по округам')
plt.show()

In [ ]:
moscow_map = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

# создаём хороплет с помощью конструктора Choropleth и добавляем его на карту
Choropleth(
    geo_data=geomap,
    data=avg_bill,
    columns=['district', 'middle_avg_bill'],
    key_on='feature.name',
    fill_color='YlOrBr',
    fill_opacity=0.8,
    legend_name='Средний чек в заведениях по районам Москвы',
).add_to(moscow_map)

# выводим карту
moscow_map

Самые высокие цены в центре и на западе Москвы. Наиболее доступные цены на Юге, Юго-Востоке и Северо-Востоке города. В остальных районах цены находятся в диапазоне 575-700 рублей.

In [ ]:
from folium.plugins.heat_map import HeatMap
m = folium.Map([55.751244, 37.618423], zoom_start=10)
heatmap_data = places[['lat','lng', 'middle_avg_bill']].copy().dropna() # без удаления пустых значений heatmap не сработает
heatmap = heatmap_data[['lat','lng', 'middle_avg_bill']] 
HeatMap(data=heatmap, radius=14).add_to(m)
m 

Наибольшая концентрация дорогих завелений находится в центре. Также можно сказать что вес точек распределен равномерно по всему городу - в каждом округе есть как и дешевые, так и заведения с высокими ценами.

**3.13 Связь среднего чека и рейтинга**

In [ ]:
#посмотрим на категории цен
places.groupby('price')['middle_avg_bill'].median()

In [ ]:
#построим диаграмму рассеяния
plt.figure(figsize=(10, 5))
sns.scatterplot(x='rating', y='middle_avg_bill', data=places)
plt.title('Корреляция между рейтингом и средним чеком') 
plt.xlabel('Рейтинг') 
plt.ylabel('Средний чек') 
plt.ylim(0,10000)
plt.show()

In [ ]:
rating_bill_corr = places['rating'].corr(places['middle_avg_bill'])
rating_bill_corr

Корреляция между средним чеком и рейтингом слабая. Можно сказать, что у заведений с высоким средним чеком реже бывают плохие отзывы. Детально проанализируем заведения с низкими рейтингами.

In [ ]:
#сделаем срез по заведениям с рейтингом ниже 3.5
#сгруппируем данные по категориям со стобцами: количество заведений в категории и средний чек
low_rating = places.query('rating < 3.5').groupby('category').agg({'name' : 'count', 'middle_avg_bill': 'median'}).reset_index()
low_rating.rename(columns={'name':'count', 'middle_avg_bill':'low_rating_bill'}, inplace=True)
low_rating

In [ ]:
a = places.groupby('category')['middle_avg_bill'].median().reset_index()
a

In [ ]:
#объединим две таблицы и выведем: категория заведения, количество заведений с рейтингом ниже 3.5,
#средний чек заведений с рейтингом ниже 3.5, средний чек по всему датасету в данной категории
low_rating_place = low_rating.merge(a, how='inner', on='category').sort_values(by='count', ascending=False).reset_index()
low_rating_place

Чаще всего заведения с низким рейтингом (ниже 3.5) встречаются среди кафе, при этом средний чек в кафе с низкой оценкой и средний чек кафе по всем данным практически не отличается - 550-560 руб. В булочных, кофейнях и пиццериях с низким рейтингом средний чек выше, чем в целом по данным категориям. В барах и пабах разница в средних чеках ощутима, практически в 8 раз. В ресторанах, фастфудах и столовых средний чек в целом выше чем средний чек в этих категориях в заведениях с низким рейтингом.

**3.13 Вывод:**


Было проанализировано  8403 объекта общественного питания Москвы. Больше всего заведений - кафе, меньше всего - булочных.

**Типы заведений:**
 Представлено 8 категорий заведений:
- кафе - самое распространенное заведение общественного питания, 28.3% от всего количество.
- рестораны - 24.3% и кофейни - 16.8%;
- бары,пабы - 9.1%;
- пиццерий и фастфудов 7.53% и 7.17% соответственно;
- меньше всего оказалось столовых и булочных - 3.75% и 3.05%.

**Посадочные места:**

В ресторанах самое большое количество мест - 86, далее идут бары и пабы  82 места в среднем, в кофейнях 80, в столовых 75 мест, в фастфудах и кафе 65 и 60 мест, и меньше всего в пиццериях и булочных - по 55 и 50 мест соответственно.

**Сетевые и несетевые заведения:**

Больше всех сетями являются булочные заведения - 61.3%, меньше всех пабы и бары - 22%. Кафе и рестораны чаще являются несетевым заведением - 67,2% и 64,3%. Кофеен почти половина сетей. 78% пабов и баров являюся нестевыми. Заведенее категории 'фастфуд' чаще несетевое также как и столовые.
Самым крупным сетевым заведением является Шоколадница - 119 кафе. Пиццерии Доминос и Додо занимают 2 и 3 места, 77 и 74 заведения соответсвенно. Самыми скромными сетями в нашем рейтинге будут Крошка Картошка и Cinnabon - по 20 точек в каждом. В общем самые распространенные сетевые категории - это кафе, кофейни, булочные, пиццерии.

**Расположение:**

В Центральном округе больше всего заведений - 2242, среди которых подавляющее число это рестораны. На Северо-Западе Москвы оказалось меньше всего заведений - 409. В остальных округах число заведений примерно одинаковое от 709 до 899.

Также выяснилось, что больше всего заведений находятся на больших улицах: шоссе, проспектах... Самая "густонаселенная" среди них это проспект Мира - 183 заведения. Пятницкая улица, хоть и небольшая по сравнению с шоссе, тем не менее вошла в наш рейтинг, там 48 заведений - наименьший показатель.

Всего на 458 улицах Москвы располагается по одному заведению общепита, кафе среди них встречаются чаще всего - 160 точек. Большинство улиц находятся в центре. Меньше всего улиц с одной точкой общественного питания в Северо-Западном и Юго-Западном округах. В остальных округах города распределение улиц с одним заведением общепита равномерное.

**Рейтинги:**

Самый высокий средний рейтинг у заведений из Центрального округа - 4.38, самый низкий из Юго-Восточного округа - 4.10. В Северном округе средняя оценка заведений - 4.24. В остальных округах рейтинг в интервале от 4.15 до 4.21.

Чаще всего заведения с низким рейтингом (ниже 3.5) встречаются среди кафе, при этом средний чек в кафе с низкой оценкой и средний чек кафе по всем данным практически не отличается - 550-560 руб. В булочных, кофейнях и пиццериях с низким рейтингом средний чек выше, чем в целом по данным категориям. В барах и пабах разница в средних чеках ощутима, практически в 8 раз. В ресторанах, фастфудах и столовых средний чек в целом выше чем средний чек в этих категориях в заведениях с низким рейтингом.

**Ценообразование:**

Самые высокие цены в центре и на западе Москвы. Наиболее доступные цены на Юге, Юго-Востоке и Северо-Востоке города. В остальных районах цены находятся в диапазоне 575-700 рублей.

**Шаг 4. Детализируем исследование: открытие кофейни**

Основателям фонда «Shut Up and Take My Money» не даёт покоя успех сериала «Друзья». Их мечта — открыть такую же крутую и доступную, как «Central Perk», кофейню в Москве. Будем считать, что заказчики не боятся конкуренции в этой сфере, ведь кофеен в больших городах уже достаточно. Попробуем определить, осуществима ли мечта клиентов.

**4.1 Сколько всего кофеен в датасете? В каких районах их больше всего, каковы особенности их расположения?**

In [ ]:
#количество кофеен
coffee = places.query('category == "кофейня"')
coffee['name'].count()

In [ ]:
coffee_distr= coffee['district'].value_counts().reset_index()
coffee_distr

In [ ]:
plt.figure(figsize=(12, 8))
sns.barplot(x='district', y='index', data=coffee_distr, color='blue')
plt.xlabel('Количество кофеен')
plt.ylabel('Округ')
plt.title('Распределение кофеен по округам')
plt.show()

In [ ]:
moscow_lat, moscow_lng = 55.751244, 37.618423

# создаём карту Москвы
coffee_map = Map(location=[moscow_lat, moscow_lng], zoom_start=10)
# создаём пустой кластер, добавляем его на карту
marker_cluster = MarkerCluster().add_to(coffee_map )

# пишем функцию, которая принимает строку датафрейма,
# создаёт маркер в текущей точке и добавляет его в кластер marker_cluster
def create_clusters(row):
    Marker(
        [row['lat'], row['lng']],
        popup=f"{row['name']} {row['rating']}",
    ).add_to(marker_cluster)

# применяем функцию create_clusters() к каждой строке датафрейма
coffee.apply(create_clusters, axis=1)

# выводим карту
coffee_map 

In [ ]:
# улицы на которых больше всего кофеен
coffee['street'].value_counts().head(15)

Всего в Москве 1413 кофеен. Больше всего заведений данной категории в Центральном округе - 428, на втором месте Север, меньше всего на Юге. Наиболее часто кофейни встречаются на длинных улицах, шоссе и проспектах, на перекрестках.

**4.2 Есть ли круглосуточные кофейни?**

In [ ]:
coffee['hours'].value_counts().sort_values(ascending=False).head(15)

In [ ]:
#сделаем срез по столбцу 'is_24/7'
coffee_24_7 = coffee[coffee['is_24/7'] == True]

map_coffee_24_7 = Map(location=[moscow_lat, moscow_lng], zoom_start=10)
# создаём пустой кластер, добавляем его на карту
marker_cluster = MarkerCluster().add_to(map_coffee_24_7)

# пишем функцию, которая принимает строку датафрейма,
# создаёт маркер в текущей точке и добавляет его в кластер marker_cluster
def create_clusters(row):
    Marker(
        [row['lat'], row['lng']],
        popup=f"{row['name']} {row['rating']}",
    ).add_to(marker_cluster)

# применяем функцию create_clusters() к каждой строке датафрейма
coffee_24_7.apply(create_clusters, axis=1)

# выводим карту
map_coffee_24_7

In [ ]:
c_24_7 = coffee_24_7['district'].value_counts().reset_index()
c_24_7.columns = ['district', 'count']
c_24_7

In [ ]:
plt.figure(figsize=(10, 7))
sns.barplot(x='count', y='district', data=c_24_7)
plt.xlabel('количество кофеен')
plt.ylabel('Округ')
plt.title('Распределение круглосуточных кофеен по округам')
plt.show()

Большинство круглосуточных кофеен расположено в центре города - 26 точек. Далее по списку Запад - 9, а за ним Юго-Запад - 7. В остальных округах крайне мало заведений от 1 до 5.  

**4.3 Какие у кофеен рейтинги? Как они распределяются по районам?**

In [ ]:
#распределение рейтингов кофеен по округам
coffee_rating = (coffee
                 .groupby('district', as_index=False)['rating'].agg('mean')
                 .round(2)
                 .sort_values(by='rating', ascending=False)
                )
coffee_rating

In [ ]:
plt.figure(figsize=(10, 7))
sns.barplot(x='rating', y='district', data=coffee_rating)
plt.xlim(4, 4.5)
plt.xlabel('Средний рейтинг')
plt.ylabel('Округ')
plt.title('Средний рейтинг кофеен по округам')
plt.show()

In [ ]:
# создаём карту Москвы
#coffee_map = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

# создаём хороплет с помощью конструктора Choropleth и добавляем его на карту
#Choropleth(
    #geo_data=geomap,
    #data=coffee_rating,
    #columns=['district', 'rating'],
    #key_on='feature.name',
    #fill_color='YlOrBr',
    #fill_opacity=0.8,
    #legend_name='Средний рейтинг кофеен по районам',
#).add_to(coffee_map)

#coffee_map

Наиболее высокие оценки имеют кофейни в Центральном и Северо-Западном округах - средняя оценка 4.33-4.34. Самый низкий рейтинг у кофеен в Западном округе - 4.2 и Северо-Восточном округе - 4.22.

**4.4 На какую стоимость чашки капучино стоит ориентироваться при открытии и почему?**

In [ ]:
#посмотрим на разброс цен на чашку капучино
coffee['middle_coffee_cup'].describe()

In [ ]:
#распределение стоимости чашки капучино по округам
coffee_price = (coffee
                 .groupby('district', as_index=False)['middle_coffee_cup'].agg('median')
                 .round(2)
                 .sort_values(by='middle_coffee_cup', ascending=False)
                )
coffee_price

In [ ]:
coffee_price_map = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

Choropleth(
    geo_data=geomap,
    data=coffee_price,
    columns=['district', 'middle_coffee_cup'],
    key_on='feature.name',
    fill_color='YlOrBr',
    fill_opacity=0.8,
    legend_name='Стоимость чашки капучино',
).add_to(coffee_price_map)

coffee_price_map

Самый дорогой капучино в Юго-Западном округе - 198 руб, в Центральном и Западном округах - по 190 руб. На Востоке Москвы самый дешевый кофе - в среднем 135 руб за чашку. В остальных округах средняя цена колеблется от 150 до 165 руб. При открытии нового заведения, для формирования стоимости чашки кофе, стоит ориентироваться на район, в котором вы открываете заведение.

**4.5 Узнаем есть ли различия в общем среднем чеке и в среднем чеке на чашку капучино в сетевых/несетевых кофейнях.**

In [ ]:
print('Средний чек в сетевых кофейнях:', coffee.query('chain == 1')['middle_avg_bill'].median())
print('Средняя стоимость чашки капучино в сетевых кофейнях:', coffee.query('chain == 1')['middle_coffee_cup'].median())

In [ ]:
print('Средний чек в несетевых кофейнях:', coffee.query('chain == 0')['middle_avg_bill'].median())
print('Средняя стоимость чашки капучино в несетевых кофейнях:', coffee.query('chain == 0')['middle_coffee_cup'].median())

Средний чек в сетевых кофейнях на 50% выше, чем в несетевых. Средняя стоимость чашки капучино чуть выше в несетевых кофейнях - 170 руб, и  155 руб в сетевых. Скорее всего это связано с тем, что сетевые заведения могут компенсировать стоимость за единицу объемами продаж.

**4.6 Посмотрим на распределение посадочных мест**

In [ ]:
coffee['seats'].describe()

In [ ]:
coffee.sample()

In [ ]:
plt.figure(figsize=(12, 5))

sort_median = (coffee.loc[:,['seats', 'district']]
    .groupby(['district'])
    .median() 
    .sort_values(by='seats')
          )


sns.boxplot(x='seats', y='district', data=coffee.query('seats<600'), order=sort_median.index)
plt.xlabel('Количество посадочных мест')
plt.ylabel('Округ')
plt.xlim()
plt.grid()
plt.title('Распределение посадочных мест в кофейнях по округам')
plt.show()

In [ ]:
#медианное количество мест
(
    coffee
.groupby('district')['seats'].agg('median')
.round(2)
.sort_values(ascending=False)
)

В среднем самые большие кофейни в Западном округе - на 96 мест. На Северо-Западе и в Центре практически одинаковое количество мест, 88 и 86 соотвественно. Наиболее скромные кофейни на Востоке и Юго-Востоке - 50-55 мест. Медианным значением по общему числу кофеен можно считать 80 мест.

**Выводы:**
Рекомендации для открытия кофейни:

- в месте расположения должна быть хорошая проходимость, транспортная доступность. Подойдут места возле метро, на больших улицах, шоссе, проспектах или в центре города;
- кофейню лучше открыть со своей концепцией, чтобы выделяться среди конкурентов;
- при открытии кофейни в местах с большой проходимостью и с ключевыми местами столицы, следует увеличить количество посадочных мест, в случае открытия в малолюдных местах, следует акцентировать внимание на эксклюзивность заведения и количество мест может быть ниже среднего.
- при ценообразовании лучше ориентироваться на среднюю цену по району.
- режим работы лучше выбрать круглосуточный и ежедневный или с 8 утра до 23 вечера;

Лучшие районы для открытия:

Центр города.
- плюсы: высокая проходимость, много туристов, достопримечательностей, ключевых мест, вокзалы, ночные клубы, музеи. Цены можно сделать выше среднего и это будет нормально, так как это самый оживленный район с большим количеством пешеходных улиц и не спит в ночное время, отлично подойдет режим работы 24/7.
- минусы: высокая стоимость аренды и огромная конкуренция (в целом большинство кофеен сконцентрировано в центральном округе).

Западный округ.
- плюсы: в этом округе находятся престижные районы города и соответственно присутствует платежеспособная публика, высокий средний чек в заведениях и низкий рейтинг поможет привлечь большее количество клиентов. Практически отсутствуют круглосуточные кофейни.
- минусы: меньше проходимость, чем в центре, престижны районы города также подразумевают высокую стоимость на аренду. Данный округ считаем наиболее перспективным для открытия новой кофейни;

Юго-Западный округ.
- плюсы: высокая средняя стоимость чашки кофе позволит иметь хорошую маржинальность, один из самых густонаселенных районов города с крупными магистралями, шоссе и улицами. Количество кофеен уже значительно меньше, следовательно меньше конкуренция.
- минусы: мало туристов, значимых мест, больше спальных районов.

ссылка на презентацию: https://disk.yandex.ru/i/m1xOR-Ky5D6PEA